In [5]:
import numpy as np
import pandas as pd

pr = pd.read_csv('data/cargo-pagerank.csv', delimiter=',');

In [6]:
#Get downloads data from crates
import requests

data = requests.get('https://crates.io/api/v1/crates?page=1&per_page=100&sort=downloads').json()['crates']
df = pd.DataFrame.from_dict(data)
df.head()

,badges,categories,created_at,description,documentation,downloads,exact_match,homepage,id,keywords,links,max_version,name,recent_downloads,repository,updated_at,versions
0,"[{'badge_type': 'travis-ci', 'attributes': {'b...",None,2015-02-03T06:17:14.147783+00:00,Random number generators and other randomness ...,https://rust-random.github.io/rand,15064017,False,https://crates.io/crates/rand,rand,None,{'version_downloads': '/api/v1/crates/rand/dow...,0.6.5,rand,3170089,https://github.com/rust-random/rand,2019-01-28T09:56:57.788327+00:00,None
1,"[{'badge_type': 'appveyor', 'attributes': {'pr...",None,2015-01-15T20:22:13.100871+00:00,Raw FFI bindings to platform libraries like li...,http://doc.rust-lang.org/libc,14978695,False,https://github.com/rust-lang/libc,libc,None,{'version_downloads': '/api/v1/crates/libc/dow...,0.2.58,libc,2027172,https://github.com/rust-lang/libc,2019-06-02T11:48:10.298304+00:00,None
2,"[{'badge_type': 'travis-ci', 'attributes': {'b...",None,2015-01-15T08:35:24.901276+00:00,A macro to generate structures which behave li...,https://docs.rs/bitflags,14011568,False,https://github.com/bitflags/bitflags,bitflags,None,{'version_downloads': '/api/v1/crates/bitflags...,1.0.4,bitflags,1999966,https://github.com/bitflags/bitflags,2019-05-02T06:02:56.939362+00:00,None
3,"[{'badge_type': 'appveyor', 'attributes': {'pr...",None,2014-11-23T01:18:10.993977+00:00,A macro for declaring lazily evaluated statics...,https://docs.rs/lazy_static,12010972,False,None,lazy_static,None,{'version_downloads': '/api/v1/crates/lazy_sta...,1.3.0,lazy_static,2018229,https://github.com/rust-lang-nursery/lazy-stat...,2019-02-27T00:02:09.802332+00:00,None
4,"[{'badge_type': 'appveyor', 'attributes': {'pr...",None,2014-12-13T22:10:19.944550+00:00,A lightweight logging facade for Rust\n,https://docs.rs/log,11992266,False,https://github.com/rust-lang/log,log,None,{'version_downloads': '/api/v1/crates/log/down...,0.4.6,log,1977182,https://github.com/rust-lang/log,2018-10-27T19:06:10.563576+00:00,None


In [7]:
len(df)

100

In [ ]:
#Compose dependency metadata data frame
df_downloads = df[['id', 'downloads']].copy()
#Rename Columns
df_downloads = df_downloads.rename(columns={'id': 'Name', 'downloads': 'Downloads'})
df_downloads.head()

In [9]:
#Load Pageranks and merge with Downloads data
df_deps = pd.read_csv('data/cargo-pagerank.csv', delimiter=',');


df_deps_downloads = pd.merge(df_deps, df_downloads, on='Name')
df_deps_downloads.head()

,ID,Name,page_rank,Downloads
0,1369785,num-traits,0.059415,8959699
1,428702,rand,0.035209,15064017
2,31533,winapi,0.033656,8867366
3,31384,serde,0.026141,10938500
4,31115,libc,0.022697,14978695


In [10]:
#Add Ranks for Pagerank and Downloads to data frame
df_deps_downloads['PR_Rank'] = df_deps_downloads.page_rank.rank(ascending= False)
df_deps_downloads['DL_Rank'] = df_deps_downloads.Downloads.rank(ascending= False)
df_deps_downloads.head()

,ID,Name,page_rank,Downloads,PR_Rank,DL_Rank
0,1369785,num-traits,0.059415,8959699,1.0,12.0
1,428702,rand,0.035209,15064017,2.0,1.0
2,31533,winapi,0.033656,8867366,3.0,13.0
3,31384,serde,0.026141,10938500,4.0,6.0
4,31115,libc,0.022697,14978695,5.0,2.0


In [11]:
#Write to osrank analysis csv
df_deps_downloads.to_csv('data/cargo-osrank-analysis.csv', index= False)

In [14]:
import numpy as np
import pandas as pd

df_contrib = pd.read_csv('data/gh-contributions.csv', delimiter=',');
df_contrib.head()

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,maintainer,repo,contributions,Name
0,1505422b2465e9a84f6fdfaa161078890c593f06@githu...,shenzhouzd/update,1140574,update
1,1a5ddd760534b32e2190eeee14c30bd038fb6adf@duane...,duaneking/rockstar_test,597893,rockstar_test
2,99b48da825c239c6ecd0a54ebfc11552d7ffb56f@mycro...,Dark5ide/mycroft-core,524381,mycroft-core
3,1505422b2465e9a84f6fdfaa161078890c593f06@local...,g0v-data/mirror,501788,mirror
4,d9075c74411dff69aa26d076c852f9974f5cf7cf@gmail...,spiderworthy/linux,495786,linux


In [15]:
df_deps_meta = pd.read_csv('data/cargo-dependencies-meta.csv', delimiter=',');
df_deps_meta.head()

,ID,Name,Platform
0,30742,acacia,Cargo
1,30745,aio,Cargo
2,30746,advapi32-sys,Cargo
3,30747,alfred,Cargo
4,30748,algebloat,Cargo


In [16]:
# Merge Metadata and GH Contributions data
df_contrib = pd.merge(df_contrib, df_deps_meta, on='Name')
df_contrib.head()

,maintainer,repo,contributions,Name,ID,Platform
0,NaN,zach14c/mysql,63816,mysql,31186,Cargo
1,35a2c6fae61f8077aab61faa4019722abf05093c@mysql...,zach14c/mysql,2740,mysql,31186,Cargo
2,marko,Anirudhk94/mysql,1989,mysql,31186,Cargo
3,marko,yslzsl/mysql,1989,mysql,31186,Cargo
4,marko,robery567/mysql,1989,mysql,31186,Cargo


In [18]:
len(df_contrib)

715099

In [19]:
#Drop Nulls and NAs
df_contrib = df_contrib.dropna()
#Drop NaNs
df_contrib = df_contrib[~df_contrib.isin([np.nan, np.inf, -np.inf]).any(1)]

In [20]:
len(df_contrib)

714977

In [21]:
#Dedup
df_contrib = df_contrib.drop_duplicates()
df_contrib = df_contrib.reset_index(drop=True)
df_contrib.head()

,maintainer,repo,contributions,Name,ID,Platform
0,35a2c6fae61f8077aab61faa4019722abf05093c@mysql...,zach14c/mysql,2740,mysql,31186,Cargo
1,marko,Anirudhk94/mysql,1989,mysql,31186,Cargo
2,marko,yslzsl/mysql,1989,mysql,31186,Cargo
3,marko,robery567/mysql,1989,mysql,31186,Cargo
4,marko,fengshao0907/mysql,1989,mysql,31186,Cargo


In [22]:
len(df_contrib)

714977

In [25]:
df_contrib

,maintainer,repo,contributions,Name,ID,Platform
0,35a2c6fae61f8077aab61faa4019722abf05093c@mysql...,zach14c/mysql,2740,mysql,31186,Cargo
1,marko,Anirudhk94/mysql,1989,mysql,31186,Cargo
2,marko,yslzsl/mysql,1989,mysql,31186,Cargo
3,marko,robery567/mysql,1989,mysql,31186,Cargo
4,marko,fengshao0907/mysql,1989,mysql,31186,Cargo
5,marko,itspluxstahre/mysql,1989,mysql,31186,Cargo
6,marko,twitter/mysql,1989,mysql,31186,Cargo
7,marko,1yfgT53/mysql,1989,mysql,31186,Cargo
8,3492c7c598251ad856af49f9b9220ecd6ff96ece@neptu...,itspluxstahre/mysql,1906,mysql,31186,Cargo
9,3492c7c598251ad856af49f9b9220ecd6ff96ece@neptu...,fengshao0907/mysql,1906,mysql,31186,Cargo


In [23]:
#Check if Nulls or NaNs
df_contrib.isnull().sum()

maintainer       0
repo             0
contributions    0
Name             0
ID               0
Platform         0
dtype: int64

In [24]:
#Write cargo contributors
df_contrib.to_csv('data/cargo-contributors.csv', index= False)